# Training ANN model on data

    Find best split number

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

In [ ]:
split_range = [4, 5, 6, 7, 8, 9, 10]


for i in split_range:
    # import data
    data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

    # 2018 data
    data = data.loc[data.index > 2018090000, :]

    # reset index
    data.reset_index(inplace = True)
    data.drop('index', axis = 1, inplace = True)

    # Divide features and labels
    X = data.iloc[:, 0:15]
    y = data.loc[:, 'Offers']
    
    X.fillna(method = 'ffill', inplace = True)
    y.fillna(method = 'ffill', inplace = True)

    X = X.round(20)
    
    # divide data into train and test with 10% test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, shuffle=False)

    # feature scaling
    sc_X = MinMaxScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
    
    def regressor_tunning(n_hidden = 5, 
                          n_neurons = 40, 
                          kernel_initializer = "he_normal",
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(Dense(units = n_neurons, input_dim = 15))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
        for layer in range(n_hidden):
            model.add(Dense(n_neurons))
            model.add(keras.layers.LeakyReLU(alpha = 0.2))
            model.add(Dropout(rate = 0.1))
        model.add(Dense(units = 1, activation = 'linear'))
        optimizer = optimizers.Adamax(lr = 0.001)
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
        return model

    tscv = TimeSeriesSplit(n_splits = i)
    hist_list = pd.DataFrame()
    count = 1
    
    regressor = regressor_tunning()
    
    for train_index, test_index in tscv.split(X_train):
          X_train_split, X_test_split = X_train[train_index], X_train[test_index]
          y_train_split, y_test_split = y_train[train_index], y_train[test_index]
          hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 100)
          hist_list = hist_list.append(hist.history, ignore_index = True)
          print(count)
          count = count + 1

    a = []
    b = []
    
    for i in range(len(hist_list.mse)):
        a.append(np.mean(hist_list.mse[i]))
        b.append(np.mean(hist_list.mae[i]))
      
    mse_cv.append(np.mean(a))
    mae_cv.append(np.mean(b))
    
    # predict for X_test  
    y_pred = regressor.predict(X_test)

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525
    
    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
    
    # smal adjustment
    y_test.replace(0, 0.0001,inplace = True)

    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)


C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/100
67/67 [==============================] - 0s 1ms/step - loss: 16166.8301 - mse: 16166.8301 - mae: 112.9569
Epoch 2/100
67/67 [==============================] - 0s 1ms/step - loss: 6360.7705 - mse: 6360.7705 - mae: 53.2657
Epoch 3/100
67/67 [==============================] - 0s 1ms/step - loss: 3871.6897 - mse: 3871.6897 - mae: 32.0483
Epoch 4/100
67/67 [==============================] - 0s 1ms/step - loss: 3940.6304 - mse: 3940.6304 - mae: 32.4088
Epoch 5/100
67/67 [==============================] - 0s 1ms/step - loss: 3785.9785 - mse: 3785.9785 - mae: 30.7988
Epoch 6/100
67/67 [==============================] - 0s 1ms/step - loss: 3770.5925 - mse: 3770.5925 - mae: 33.0115
Epoch 7/100
67/67 [==============================] - 0s 1ms/step - loss: 3934.6543 - mse: 3934.6543 - mae: 32.3982
Epoch 8/100
67/67 [==============================] - 0s 1ms/step - loss: 3852.8740 - mse: 3852.8740 - mae: 31.9605
Epoch 9/100
67/67 [==============================] - 0s 1ms/step - loss: 3872

67/67 [==============================] - 0s 1ms/step - loss: 3677.1194 - mse: 3677.1194 - mae: 29.8394
Epoch 72/100
67/67 [==============================] - 0s 1ms/step - loss: 3649.7561 - mse: 3649.7561 - mae: 30.0647
Epoch 73/100
67/67 [==============================] - 0s 1ms/step - loss: 3710.0432 - mse: 3710.0432 - mae: 30.2029
Epoch 74/100
67/67 [==============================] - 0s 1ms/step - loss: 3705.4666 - mse: 3705.4666 - mae: 31.0047
Epoch 75/100
67/67 [==============================] - 0s 1ms/step - loss: 3653.8008 - mse: 3653.8008 - mae: 29.9646
Epoch 76/100
67/67 [==============================] - 0s 1ms/step - loss: 3666.0195 - mse: 3666.0195 - mae: 29.3199
Epoch 77/100
67/67 [==============================] - 0s 1ms/step - loss: 3721.7656 - mse: 3721.7656 - mae: 30.2001
Epoch 78/100
67/67 [==============================] - 0s 1ms/step - loss: 3632.1589 - mse: 3632.1589 - mae: 30.0555
Epoch 79/100
67/67 [==============================] - 0s 1ms/step - loss: 3709.6995 -

133/133 [==============================] - 0s 1ms/step - loss: 2620.6165 - mse: 2620.6165 - mae: 28.5621
Epoch 41/100
133/133 [==============================] - 0s 3ms/step - loss: 2635.7090 - mse: 2635.7090 - mae: 28.3881
Epoch 42/100
133/133 [==============================] - 0s 2ms/step - loss: 2598.2422 - mse: 2598.2422 - mae: 28.8177
Epoch 43/100
133/133 [==============================] - 0s 1ms/step - loss: 2684.6667 - mse: 2684.6667 - mae: 28.9119
Epoch 44/100
133/133 [==============================] - 0s 2ms/step - loss: 2606.5110 - mse: 2606.5110 - mae: 28.0101
Epoch 45/100
133/133 [==============================] - 0s 2ms/step - loss: 2594.1042 - mse: 2594.1042 - mae: 28.3533
Epoch 46/100
133/133 [==============================] - 0s 1ms/step - loss: 2594.9553 - mse: 2594.9553 - mae: 28.1310
Epoch 47/100
133/133 [==============================] - 0s 1ms/step - loss: 2671.8513 - mse: 2671.8513 - mae: 28.3739
Epoch 48/100
133/133 [==============================] - 0s 1ms/step -

200/200 [==============================] - 0s 2ms/step - loss: 2193.0928 - mse: 2193.0928 - mae: 27.7587
Epoch 10/100
200/200 [==============================] - 0s 1ms/step - loss: 2180.5405 - mse: 2180.5405 - mae: 27.5795
Epoch 11/100
200/200 [==============================] - 0s 1ms/step - loss: 2169.4749 - mse: 2169.4749 - mae: 27.8263
Epoch 12/100
200/200 [==============================] - 0s 1ms/step - loss: 2114.4077 - mse: 2114.4077 - mae: 27.4367
Epoch 13/100
200/200 [==============================] - 0s 1ms/step - loss: 2154.0762 - mse: 2154.0762 - mae: 27.6706
Epoch 14/100
200/200 [==============================] - 0s 1ms/step - loss: 2185.8662 - mse: 2185.8662 - mae: 27.9374
Epoch 15/100
200/200 [==============================] - 0s 1ms/step - loss: 2159.2402 - mse: 2159.2402 - mae: 27.6683
Epoch 16/100
200/200 [==============================] - 0s 1ms/step - loss: 2171.3567 - mse: 2171.3567 - mae: 27.6281
Epoch 17/100
200/200 [==============================] - 0s 1ms/step -

...

152/152 [==============================] - 0s 2ms/step - loss: 2376.6318 - mse: 2376.6318 - mae: 27.4434
Epoch 18/100
152/152 [==============================] - 0s 2ms/step - loss: 2341.0574 - mse: 2341.0574 - mae: 26.7323
Epoch 19/100
152/152 [==============================] - 0s 2ms/step - loss: 2382.4812 - mse: 2382.4812 - mae: 27.3907
Epoch 20/100
152/152 [==============================] - 0s 2ms/step - loss: 2381.5825 - mse: 2381.5825 - mae: 27.1255
Epoch 21/100
152/152 [==============================] - 0s 2ms/step - loss: 2382.0771 - mse: 2382.0771 - mae: 27.6539
Epoch 22/100
152/152 [==============================] - 0s 2ms/step - loss: 2388.6001 - mse: 2388.6001 - mae: 27.0447
Epoch 23/100
152/152 [==============================] - 0s 2ms/step - loss: 2353.6772 - mse: 2353.6772 - mae: 27.2149
Epoch 24/100
152/152 [==============================] - 0s 2ms/step - loss: 2338.0708 - mse: 2338.0708 - mae: 26.9168
Epoch 25/100
152/152 [==============================] - 0s 2ms/step -

152/152 [==============================] - 0s 2ms/step - loss: 2322.5837 - mse: 2322.5837 - mae: 26.9766
Epoch 86/100
152/152 [==============================] - 0s 2ms/step - loss: 2330.3618 - mse: 2330.3618 - mae: 27.0039
Epoch 87/100
152/152 [==============================] - 0s 3ms/step - loss: 2299.5554 - mse: 2299.5554 - mae: 26.6812
Epoch 88/100
152/152 [==============================] - 0s 2ms/step - loss: 2310.9236 - mse: 2310.9236 - mae: 26.6383
Epoch 89/100
152/152 [==============================] - 0s 2ms/step - loss: 2316.5293 - mse: 2316.5293 - mae: 26.8112
Epoch 90/100
152/152 [==============================] - 0s 2ms/step - loss: 2371.7861 - mse: 2371.7861 - mae: 27.1394
Epoch 91/100
152/152 [==============================] - 0s 3ms/step - loss: 2339.3259 - mse: 2339.3259 - mae: 26.8089
Epoch 92/100
152/152 [==============================] - 0s 3ms/step - loss: 2320.9006 - mse: 2320.9006 - mae: 26.9313
Epoch 93/100
152/152 [==============================] - 0s 3ms/step -

182/182 [==============================] - 0s 2ms/step - loss: 2049.3108 - mse: 2049.3108 - mae: 25.8715
Epoch 54/100
182/182 [==============================] - 0s 2ms/step - loss: 2099.0012 - mse: 2099.0012 - mae: 25.8737
Epoch 55/100
182/182 [==============================] - 1s 3ms/step - loss: 2124.3354 - mse: 2124.3354 - mae: 26.1142
Epoch 56/100
182/182 [==============================] - 0s 2ms/step - loss: 2087.4336 - mse: 2087.4336 - mae: 25.8207
Epoch 57/100
182/182 [==============================] - 0s 2ms/step - loss: 2050.1521 - mse: 2050.1521 - mae: 25.6298
Epoch 58/100
182/182 [==============================] - 0s 2ms/step - loss: 2054.5537 - mse: 2054.5537 - mae: 25.4182
Epoch 59/100
182/182 [==============================] - 0s 2ms/step - loss: 2103.1211 - mse: 2103.1211 - mae: 26.3709
Epoch 60/100
182/182 [==============================] - 0s 3ms/step - loss: 2054.0334 - mse: 2054.0334 - mae: 25.8906
Epoch 61/100
182/182 [==============================] - 1s 3ms/step -

212/212 [==============================] - 1s 2ms/step - loss: 2020.0638 - mse: 2020.0638 - mae: 26.5697
Epoch 23/100
212/212 [==============================] - 0s 2ms/step - loss: 2002.3452 - mse: 2002.3452 - mae: 26.2504
Epoch 24/100
212/212 [==============================] - 0s 2ms/step - loss: 2013.2948 - mse: 2013.2948 - mae: 26.3472
Epoch 25/100
212/212 [==============================] - 0s 2ms/step - loss: 2017.0941 - mse: 2017.0941 - mae: 26.0720
Epoch 26/100
212/212 [==============================] - 0s 2ms/step - loss: 1987.0511 - mse: 1987.0511 - mae: 26.1505
Epoch 27/100
212/212 [==============================] - 0s 2ms/step - loss: 2000.9200 - mse: 2000.9200 - mae: 26.3395
Epoch 28/100
212/212 [==============================] - 0s 2ms/step - loss: 2001.1364 - mse: 2001.1364 - mae: 26.3053
Epoch 29/100
212/212 [==============================] - 0s 2ms/step - loss: 1973.1356 - mse: 1973.1356 - mae: 25.8142
Epoch 30/100
212/212 [==============================] - 0s 2ms/step -

212/212 [==============================] - 1s 3ms/step - loss: 1958.2616 - mse: 1958.2616 - mae: 25.7211
Epoch 91/100
212/212 [==============================] - 0s 2ms/step - loss: 1917.3016 - mse: 1917.3016 - mae: 25.8249
Epoch 92/100
212/212 [==============================] - 0s 2ms/step - loss: 1987.1866 - mse: 1987.1866 - mae: 25.8418
Epoch 93/100
212/212 [==============================] - 0s 2ms/step - loss: 1971.7566 - mse: 1971.7566 - mae: 25.8588
Epoch 94/100
212/212 [==============================] - 0s 2ms/step - loss: 1969.9865 - mse: 1969.9865 - mae: 25.7556
Epoch 95/100
212/212 [==============================] - 0s 2ms/step - loss: 1968.8694 - mse: 1968.8694 - mae: 25.6435
Epoch 96/100
212/212 [==============================] - 1s 2ms/step - loss: 1983.8816 - mse: 1983.8816 - mae: 25.8531
Epoch 97/100
212/212 [==============================] - 0s 2ms/step - loss: 1938.3649 - mse: 1938.3649 - mae: 25.5406
Epoch 98/100
212/212 [==============================] - 0s 2ms/step -

In [ ]:
data.head()

In [ ]:
rmse_cv = []
for i in mse_cv:
    rmse_cv.append(i ** 0.5)
    
results = pd.DataFrame({'rmse_cv':rmse_cv,
              
                        'mae_cv': mae_cv,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor}, index = split_range)

results

In [ ]:
!pip install matplotlib

%matplotlib notebook

import matplotlib.pyplot as plt

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged RMSE on all test set \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6], y = rmse_gen, label = 'Overall error')
plt.legend(loc = 'lower right')
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6], split_range)
plt.tight_layout()
plt.savefig('Split_number_all_RMSE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged RMSE on spike regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6], y =rmse_spi, label = 'Spike regions', color = 'orange')
plt.legend(loc = 'lower right')
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6], split_range)
plt.tight_layout()
plt.savefig('Split_number_spike_RMSE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged RMSE on normal regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6], y =rmse_nor, label = 'Normal regions', color = 'green')
plt.legend(loc = 'lower right')
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6], split_range)
plt.tight_layout()
plt.savefig('Split_number_nomal_RMSE.png')

In [ ]:
plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged MAE on all test set \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6], y = mae_gen, label = 'Overall error')
plt.legend(loc = 'lower right')
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6], split_range)
plt.tight_layout()
plt.savefig('Split_number_all_MAE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged MAE on spike regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6], y =mae_spi, label = 'Spike regions', color = 'orange')
plt.legend(loc = 'lower right')
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6], split_range)
plt.tight_layout()
plt.savefig('Split_number_spike_MAE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged MAE on normal regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6], y = mae_nor, label = 'Normal regions', color = 'green')
plt.legend(loc = 'lower right')
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6], split_range)
plt.tight_layout()
plt.savefig('Split_number_normal_MAE.png')